In [92]:
import pickle
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

from sklearn_extra.cluster import KMedoids
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

import torch
import re

In [84]:
hours_back = 8

section_number = 100

current_time = datetime.now()

start_time = current_time - timedelta(hours=hours_back)

valid_hours = []
for hour in range(1, hours_back + 1):  # 0부터 hours_back까지의 시간을 계산
    valid_hour = (start_time - timedelta(hours=hour)).strftime('%H')
    valid_hours.append(valid_hour)

file_patterns = [f"./crawling_data/{section_number}/_{hour}.pkl" for hour in valid_hours]

merged_documents = []
for file_pattern in file_patterns:
    if(os.path.isfile(file_pattern)):
        with open(file_pattern, 'rb') as file:
            merged_documents += pd.read_pickle(file)
            print(len(merged_documents))
            print(file_pattern)
print(f"문서의 개수: {len(merged_documents)} 개 입니다.")

print(file_patterns)

merged_documents.remove(merged_documents[12])
merged_documents.remove(merged_documents[29])
merged_documents.remove(merged_documents[27])
merged_documents.remove(merged_documents[38])
merged_documents.remove(merged_documents[35])


14
./crawling_data/100/_08.pkl
28
./crawling_data/100/_07.pkl
36
./crawling_data/100/_06.pkl
39
./crawling_data/100/_05.pkl
42
./crawling_data/100/_04.pkl
46
./crawling_data/100/_03.pkl
문서의 개수: 46 개 입니다.
['./crawling_data/100/_10.pkl', './crawling_data/100/_09.pkl', './crawling_data/100/_08.pkl', './crawling_data/100/_07.pkl', './crawling_data/100/_06.pkl', './crawling_data/100/_05.pkl', './crawling_data/100/_04.pkl', './crawling_data/100/_03.pkl']


In [101]:
data = []
for doc in merged_documents:
    data.append(doc.get("content"))

countvectorizer = CountVectorizer(stop_words=["\n", "기자", "뉴스"], ngram_range=(1,2), max_df = 0.4, min_df = 0.05, lowercase=True, max_features = 550)
data = TfidfTransformer().fit_transform(countvectorizer.fit_transform(data))

clst = KMedoids(n_clusters=4)
clst.fit(data)

medoid_indices = clst.medoid_indices_
print(f"중심으로 선정된 기사들의 index: {medoid_indices}")

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')
summarizer = pipeline('summarization', model='gogamza/kobart-summarization')

for med in medoid_indices:
    text = merged_documents[med].get("content").replace("\n"," ")
    re.sub('[^A-Za-z0-9가-힣]', ' ', text)
    print("원문: ")
    print(text)
    while len(text) > 1024:
        outs = []   
        for chunk in get_overlapped_chunks(text, 256, 32):
            print("chunk: ")
            print(chunk)
            out = summarizer(chunk, max_length=64, min_length=8)
            outs.append(out[0]['summary_text'])
        text = '\n'.join(outs)

    # text = summarizer('\n'.join(outs), max_length=128, min_length=32)[0]['summary_text']
    print(text)
    # raw_input_ids = tokenizer.encode(text)
    # print(raw_input_ids)
    # input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
    # input_ids = torch.tensor([input_ids])
    # long_summary_text_ids = model.generate(
    #     input_ids = input_ids,
    #     bos_token_id = model.config.bos_token_id,
    #     eos_token_id = model.config.eos_token_id,
    #     length_penalty=1, # 길이에 대한 penalty 값. 1보다 작은 경우 더 짧은 문장을 생성하도록 유도하며, 1보다 클 경우 길이가 더 긴 문장을 유도
    #     max_length = 256, # 요약문의 최대 길이 설정
    #     min_length = 50, # 요약문의 최소 길이 설정
    #     num_beams = 8) # 문장 생성 시 다음 단어를 탐색하는 영역의 개수
    # longBriefing = tokenizer.decode(long_summary_text_ids[0], skip_special_tokens=True)
    # print(longBriefing)
    print("---------------------------------")







중심으로 선정된 기사들의 index: [23  7  9  8]
원문: 
윤석열 대통령의 '채상병 특검법' 4번째 거부권 행사가 예상되는 가운데, 개혁신당 허은아 대표가 국민의힘 한동훈 대표를 향해 "독대, 독대 외칠 시간에, 채상병 특검법에 거부권을 행사하지 말라고 대통령에게 공개적으로 요청하라"고 직격했다.   허 대표는 26일 최고위원회의에서 "(채 상병이 사망하지 않았다면) 정상적이었다면 오늘은 '해병 1292기 채수근 병장'이 1천여 명의 동기들과 함께 전역했을 날"이라고 지적한 후 "'(윤 대통령이 만찬에서) 고기반찬 준비할 시간에 민심부터 살피자'고 여당 대표로서 할 말을 왜 못 하시는 건가"라며 이같이 촉구했다.   허 대표는 "여당은 대통령과 독대를 하네 마네, 대통령실에서 거절했네 마네 하는 뉴스로 며칠째 떠들고 있다"며 "국민은 피곤하기만 하다. 할 말이 있으면 면전에서 당당하게 하면 될 것을, 독대가 뭐 그리 중요한가. 그럴 용기도 없이 무슨 당대표를 한다고 하나"라고 비판했다.   허 대표는 "정상적이었다면 오늘 대한민국 청년 채수근은 예비군 마크를 달고 가족들과 평범한 저녁 식사를 하고 있을 것이다. 이러한 정상과 평범을 깨뜨린 것은 비정상과 무책임"이라고 지적했고 "대민 지원작전이 정상적으로 이루어졌다면, 안전장구 하나 없이 병사들을 급류 속에 밀어 넣는 일은 없었을 것"이라고 지적했다.   허 대표는 이어 "대통령의 격노와 상관없이 해병대 수사대 조사 결과는 경찰에 이첩되었을 것이고, 정상적이었다면, 진실을 밝히라는 특검법을 만들 이유 또한 없었을 것"이라며 "정상적이었다면 해병대 수사단장이 항명 수괴죄로 법정을 드나드는 이상한 사태는 벌어지지 않았을 것"이라고도 지적했다.   허 대표는 "지금 우선순위가 되어야 하는 일은 채상병 사건의 진실을 밝히고, 책임자를 엄중히 처벌해 청년들이 자랑스럽게 복무할 수 있는 정상적인 군대를 만드는 것"이라며 "그것이야말로 국가 안보의 첫걸음이며, 병역 의무를 다하는 군인들에 대한 최소한의 존중"이

Your max_length is set to 64, but you input_length is only 59. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


chunk: 
군대를 만드는 것"이라며 "그것이야말로 국가 안보의 첫걸음이며, 병역 의무를 다하는 군인들에 대한 최소한의 존중"이라고 했다.           ▲개혁신당 허은아 대표가 26일 오전 국회에서 열린 취임 100일 기자회견에서 취재진의 질문에 답하고 있다. ⓒ연합뉴스
26일 최고위원 최고위원회의에서 '채상병 특검법' 4번째 거부권 행사가 예상되는 가운데, 개혁신당 허은아 대표가 국민의힘 한동훈 대표를 향해 "독대, 독대 외칠 시간에, 채상병 특검법에 거부권을 행사하지 말라고 대통령에게 공개적으로 요청하라고 대통령에게 공개적으로 요청하라고 요청
허 대표는 대표는 "여당은 대통령과 독대를 하네 마네, 대통령실에서 거절했네 마네 하는 뉴스로 며칠째 떠들고 있다"며 "국민은 피곤하기만 하다. 할 말이 있으면 면전에서 당당하게 하면 될 것을, 독대가 뭐 그리 중요한가. 그럴 용기도 없이 무슨 당대표를 한다고 하나
허 대표는 대표는 "정상적이었다면 오늘 대한민국 청년 채수근은 예비군 마크를 달고 가족들과 평범한 저녁 식사를 하고 있을 것이다.   허 대표는 " 정상과 평범을 깨뜨린 것은 비정상과 무책임"이라고 지적했고 "대민 지원작전이 정상적으로 이루어졌다면, 안전장구 하나 없이
허 대표는 대표는 "정상적이었다면 해병대 수사대 조사 결과는 경찰에 이첩되었을 것이고, 정상적이었다면, 진실을 밝히라는 특검법을 만들 이유 또한 없었을 것"이라며 "   허 대표는 지금 우선순위가 되어야 하는 일은 채상병 사건의 진실을 밝히고, 책임자를 엄중히 처벌해 청년들이
26일                                                              
---------------------------------
원문: 
[the300] 김여정 "지난 23일 부산항에 '美 이상물체'(버몬트함) 입항 포착" 주장 만리경-1호, 그때 부산 상공 지나쳤지만…"사진 해상도 잠수함 구분할 정도 안 돼"         북한 김정은의 여동생인 김여정 북한 노

Your max_length is set to 64, but you input_length is only 51. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


chunk: 
 뉴스1. All rights reserved. 무단 전재 및 재배포,  AI학습 이용 금지. /사진=(서울=뉴스1) 김민지 김민지 김민지 김민지 기자          
chunk: 
                                                              
방송문화진흥회법 개정안은 방송문화진흥회법 개정안은 방송문화진흥회법 개정안은 방송문화진흥회법 개정안은 방송문화진흥회법 개정안은 방송문화진흥회법 개정안은 방송문화진흥회법 개정안은 방송문화진흥회법 개정안은 방송문화진흥회법 개정안은
방송문화진흥회법 개정안은 방송문화진흥회법 개정안은 방송문화진흥회법 개정안은 방송문화진흥회법 개정안은 방송문화진흥회법 개정
chunk: 
법 개정안은 방송문화진흥회법 개정안은 방송문화진흥회법 개정안은 방송문화진흥회법 개정안은 방송문화진흥회법 개정안은 방송문화진흥회법 개정안은 방송문화진흥회법 개정안은
지원법은 전 국민에게 지역사랑상품권을 지급하고, 국가와 지방자치단체가 민생회복지원금 지급에 필요한 행정·재정적 지원을 하도록 하는 지원법은 전 국민에게 지역사랑상품권을 지급하고, 국가와 지방자치단체가 민생회복지원금 지급에 필요한 행정·재정적 지원을 하도록 하는 내용으로, 지원 금액은 지급 대상에
chunk: 
적 지원을 하도록 하는 내용으로, 지원 금액은 지급 대상에
25만원 지원법 역시 예산당국의 예산편성권을 훼손하고 재정건전성 악화 등을 부추길 것이라는 게 정부의 판단하에 정부의 판단하에 부결된된 국회 부결법안이 모두 부결된 후 민주당, 조국혁신당, 진보당 등은 본회의장 밖으로 나가 국회 로텐더홀에서 "민
방송4법 재표결 부결을 알리고 '방송4법' 재표결 부결을 알리고  방송사법 재표결 부결을 알리고하여 방송사법 재표결 부결을 알리고하여 방송4법·노란봉투법
chunk: 
하여 방송사법 재표결 부결을 알리고하여 방송4법·노란봉투법·민생회복지원금법' 재표결 부결을
딥페이크 성범죄 처벌법은 딥페이크 성착취물을 소지하거나 시청할 경우 3년 이하

Your max_length is set to 64, but you input_length is only 49. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


chunk: 
의원은 "증인 신청 협상을 하다보면 이유도 논리도 없는 이름들이 많이 등장하는데 야당은 아예 들을 생각조차 하지 않는다"며 "이번 국감도 결국 정쟁으로 끝날 게 분명해보이는 만큼 국민들께 죄송하다"고 미리 사과했다.
chunk: 
 국회            지난 4일 오전 국회 법제사법위원회 전체회의에서 정청래 법사위원장이 국민의힘 의원들의 불참했음에도 회의를 개의하기 위해 의사봉을 두드리고 의사봉을 두드리고 의사봉을 두드리고 있다. 
26일 정치권에 따르면 국회 법제사법위원회·행정안전위원회·행정안전위원회·과학기술정보방송통신위원회·문화체육관광위원회·기획재정위원회·산업통상자원중소벤처기업위원회·산업통상자원중소벤처기업위원회 등 6개 상임위원회로 구성된 피감기관 감사를 위해 여야 합의로 
chunk: 
개 상임위원회로 구성된 피감기관 감사를 위해 여야 합의로 의결돼야 할
국회는감감기관에 대한 감사 일정과 출석할 증인과 참고인을 정하게 되면 사실상 해당 상임위의 국정감사 준비는 마무리 된 것이나 다름없다.
'협의 협의가 되지 않은 명단을 야권이 일방적으로 밀어붙이는 경우가 허다한 '협의가 되지 않은 명단을 야권이 일방적으로 밀어붙이는 경우가 허다한 '협의가 되지 않은 명단을 야권이 일방적으로 밀어붙이는 경우가 허다한 '협의가 되지 않은 명단을 야권이 일방
chunk: 
이는 경우가 허다한 '협의가 되지 않은 명단을 야권이 일방적으로 일방적으로 밀어붙이는 경우가
윤석열 대통령 부인인 김건희 여사를 증인으로 채택했다.
지난 8월 7일 오전 서울 여의도 국회에서 열린 과학기술정보방송통신위원회 전체회의에서 최민희 과방위원장이 여당 의원들의 불참 속에 안건을 처리하며 의사봉을 두드리고 의사봉을 두드리고 있다. c c c c c c c c c c c c c c c c c c c c c c c
민주당은 법사위와 마찬가지로 행안위에서
chunk: 
 c c c c c
민주당은 법사위와 마찬가지로 행안위에서도 공천개입 의혹 관련자인 김 전 의원과 명 씨를 증인으로 채택하

Your max_length is set to 64, but you input_length is only 7. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


chunk: 
들께 죄송하다는 미리 사과했다.
지난 4일 국회 법제사법위원회 전체회의에서 정청래 법사위원장이 국민의힘 의원들의 불참했음에도 회의를 개의하기 위해 의사봉을 두드리고 의사봉을 두드리고 의사봉을 두드리고 의사봉을 두드리고 의사봉을 두드리고 의사봉을 두드리고 의사봉을 두드리고 의사봉을 두드
 협의감감기관 감사를 위해 여야 합의로 의결돼야 할국회는감감기관에 대한 감사 일정과 출석할 증인과 참고인을 정하게 되면 사실상 해당 상임 상임위의 국정감사 준비는 마무리 된 것이나 다름 '불'협의 협의가 되지 않은 명단을 야권이 일방적으로 밀어붙이는 경우가 허다한
 협의가 되지 않은 명단을 야권이 일방적으로 일방적으로 일방적으로 밀어붙이는 경우가 허다한 '협의가 되지 않은 명단을 야권이 일방적으로 일방적으로 일방적으로 일방적으로 밀어붙이는 경우가 허다한 '협의가 되지 않은 명단을 야권이 일방적으로 일방적으로 일방적으로 일방적으로 일방적으로 밀어붙이는 경우가외윤석열 대통령 부인인 김건
 c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c
명단을 확정하는데 명단을 확정하는데 야당의 일방적인 주장만이 반영되고 있는 명단을 확정하는데 야당의 일방적인 주장만이 반영되고 있단 점으로 인해 명단을 확정하는데 명단을 확정하는데 야당의 일방적인 주장만이 반영되고 있단 점으로 인해 명단을 확정하는데 명단을 확정하는데 야당의 일방적인
들이들께 죄송 죄송하다는 미리 사과했다.
---------------------------------


In [95]:
from transformers import pipeline


def get_overlapped_chunks(text, chunk, overlap):
    return [text[a:a+chunk] for a in range(0, len(text), chunk-overlap)]



In [88]:
merged_documents[9]

{'title': '"사기 당했다" 여야 충돌···그래도 \'성착취 딥페이크 처벌법\'은 통과',
 'link': 'https://n.news.naver.com/mnews/article/008/0005094419',
 'content': '[the300] \'거부권 행사\' 방송4법·노란봉투법·25만원 지원법은 재표결 후 \'부결·폐기\'\n\n\n\n\n\n\n\n\n[서울=뉴시스] 조성우 기자 = 추경호 국민의힘 원내대표가 26일 오후 서울 여의도 국회에서 열린 제418회국회(정기회) 제9차 본회의에서 직원이 방송 4법과 \'전 국민 25만원 지원법\'(민생회복지원금지급 특별조치법), \'노란봉투법\'(노동조합·노동관계조정법 개정안)에 재의의 건 표결 진행 중 우원식 국회의장을 찾아가 대화하고 있다. 2024.09.26. xconfind@newsis.com /사진=조성우\n26일 국회 본회의는 시작부터 여야가 거세게 충돌했다. 국가인권위원회 상임위원 여당 몫 추천인사 선출안이 부결되자 여당은 "사기 당할 줄 몰랐다"고 야당을 성토했고, 야당은 여당을 향해 "윤석열 정권의 인사 문제는 용납할 수 없다"고 맞받았다.\n그럼에도 불구하고 여당 의원들은 정회 후 회의장에 돌아와 딥페이크(인공지능을 활용한 합성 영상물) 성착취물 소지·시청에 대해 처벌하는 내용을 골자로 한 법안 등 약속됐던 민생 법안 처리에 동참했다. 대통령 재의요구권(거부권)이 행사돼 국회로 돌아간 방송4법, 노란봉투법, 25만원 지원법은 모두 부결 폐기됐다.\n━\n방송4법·노란봉투법·25만원 지원법, 줄줄이 폐기···여 \'박수\' 야 "집권여당의 민생포기"\n━\n이날 서울 여의도 국회 본회의에서 △방송4법(방송통신위원회의 설치 및 운영에 관한 법률·방송법·방송문화진흥회법·한국교육방송공사법 등 개정안) △노란봉투법(노동조합 및 노동관계조정법 개정안) △25만원 지원법(민생회복지원지급 특별조치법) 등 6개 법안이 모두 모두 부결 폐기됐다.\n이날 방송통신위원회의 설치 및 운영에 관한 법률 개정